# ETL (Extracción, Transformación y Limpieza General)

Al revisar los datos limpios, se observa que se deben corregir algunos datos, al tratarlos de abrir en power bi, se determina que es más efectivo cambiar la estructura de la información evitando las uniones de datos en la herramienta de visualización.

A continuación se siguen analizando la estructura de los datos y transformaciones necesarias:

#### 1. Importar librerías:

In [42]:
import pandas as pd                                                           # Para abrir y transformar los archivos
import numpy as np                                                            # Cálculos y manipulación de datos

#### 2. Abrir archivos relacionados a los siniestros viales:

In [43]:
H_Homicidios = pd.read_csv('Datos_limpios\Hechos_Homicidios_CABA.csv')        # Datos de Hechos de Homicidios
V_Homicidios = pd.read_csv('Datos_limpios\Victimas_Homicidios_CABA.csv')      # Datos de Víctimas de Homicidios
H_Lesiones = pd.read_csv('Datos_limpios\Hechos_Lesiones_CABA.csv')            # Datos de Hechos de Lesiones
V_Lesiones = pd.read_csv('Datos_limpios\Victimas_Lesiones_CABA.csv')          # Datos de Víctimas de Lesiones
Poblacion = pd.read_csv('Datos_limpios\Población_CABA.csv')                   # Datos de Población

In [44]:
# Revisar que las columnas sean iguales, para Hechos en Lesiones y Homicidios

print(H_Homicidios.columns)
print(H_Lesiones.columns)

Index(['ID', 'FECHA', 'HORA', 'TIPO_DE_CALLE', 'COMUNA', 'pos x', 'pos y',
       'VICTIMA', 'ACUSADO'],
      dtype='object')
Index(['id', 'fecha', 'hora', 'comuna', 'tipo_calle', 'longitud', 'latitud',
       'victima', 'acusado'],
      dtype='object')


Revisar si los datos están en el mismo periódo de análisis:

In [45]:
print('El archvio de Homicidios tiene registros desde ', H_Homicidios['FECHA'].min(),' hasta ', H_Homicidios['FECHA'].min())
print('El archvio de Homicidios tiene registros desde ', H_Lesiones['fecha'].min(),' hasta ', H_Lesiones['fecha'].min())

El archvio de Homicidios tiene registros desde  2016-01-01  hasta  2016-01-01
El archvio de Homicidios tiene registros desde  2019-01-01  hasta  2019-01-01


Aunque se unirán los dataframes, se tendrá presente que los análisis que involucren Lesiones deben realizarse solo para los años 2019 al 2021

#### 3. Unir los dataframe de "Hechos" para "Homicidios" y "Lesiones":

Se llega a la conclusión que es mejor manejar un archivo grande para Hechos y otro para Víctimas, esto no se realizó en el ETL porque en los db se podían gestionar las uniones.

In [46]:
# Agregar columna "TIPO" a cada DataFrame
H_Homicidios['TIPO'] = 'HOMICIDIO'
H_Lesiones['TIPO'] = 'LESION'

# Renombrar columnas en H_Lesiones para que coincidan con H_Homicidios
H_Lesiones = H_Lesiones.rename(columns={
    'id': 'ID',
    'fecha': 'FECHA',
    'hora': 'HORA',
    'comuna': 'COMUNA',
    'tipo_calle': 'TIPO_DE_CALLE',
    'longitud': 'pos x',
    'latitud': 'pos y',
    'victima': 'VICTIMA',
    'acusado': 'ACUSADO'
})

# Reorganizar las columnas en H_Homicidios
H_Homicidios = H_Homicidios[['ID', 'FECHA', 'HORA', 'COMUNA','TIPO_DE_CALLE', 'pos x', 'pos y', 'VICTIMA', 'ACUSADO', 'TIPO']]

# Concatenar verticalmente los DataFrames
Hechos = pd.concat([H_Homicidios, H_Lesiones], ignore_index=True)


In [47]:
Hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24481 entries, 0 to 24480
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             24481 non-null  object
 1   FECHA          24481 non-null  object
 2   HORA           24481 non-null  object
 3   COMUNA         24481 non-null  object
 4   TIPO_DE_CALLE  24481 non-null  object
 5   pos x          24481 non-null  object
 6   pos y          24481 non-null  object
 7   VICTIMA        24481 non-null  object
 8   ACUSADO        24481 non-null  object
 9   TIPO           24481 non-null  object
dtypes: object(10)
memory usage: 1.9+ MB


#### 4. Arreglar formato de fecha y valores "SD" como nulos para poderlos administrar en el Dashborad:

In [48]:
# Reemplazar "SD" con NaN en ambos DataFrames
Hechos.replace('SD', np.nan, inplace=True)

# Convertir la columna 'FECHA' al tipo datetime
Hechos['FECHA'] = pd.to_datetime(Hechos['FECHA'], errors='coerce')

Hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24481 entries, 0 to 24480
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             24481 non-null  object        
 1   FECHA          24481 non-null  datetime64[ns]
 2   HORA           24476 non-null  object        
 3   COMUNA         23022 non-null  object        
 4   TIPO_DE_CALLE  13436 non-null  object        
 5   pos x          24479 non-null  object        
 6   pos y          24479 non-null  object        
 7   VICTIMA        13739 non-null  object        
 8   ACUSADO        9170 non-null   object        
 9   TIPO           24481 non-null  object        
dtypes: datetime64[ns](1), object(9)
memory usage: 1.9+ MB


#### 4.1. corregir formato de coordenadas:

In [49]:
Hechos['pos y'] = Hechos['pos y'].replace("inf", np.nan)

# Corregir los valores para el dato con "ID" 24480
Hechos.loc[Hechos['ID'] == 'LC-2020-0244615', 'pos x'] = -58.29963273
Hechos.loc[Hechos['ID'] == 'LC-2020-0244615', 'pos y'] = -34.72890716

# Convertir la columna 'pos x' a texto
Hechos['pos x'] = Hechos['pos x'].astype(float)
Hechos['pos y'] = Hechos['pos y'].astype(float)

# Reemplazar valores infinitos por NaN en las columnas 'pos x' y 'pos y'
Hechos['pos x'] = Hechos['pos x'].replace([np.inf, -np.inf], np.nan)
Hechos['pos y'] = Hechos['pos y'].replace([np.inf, -np.inf], np.nan)

# Mostrar el DataFrame actualizado
Hechos

,ID,FECHA,HORA,COMUNA,TIPO_DE_CALLE,pos x,pos y,VICTIMA,ACUSADO,TIPO
0,2016-0001,2016-01-01,04:00:00,8,AVENIDA,-58.475340,-34.687570,MOTO,AUTO,HOMICIDIO
1,2016-0002,2016-01-02,01:15:00,9,GRAL PAZ,-58.508775,-34.669777,AUTO,PASAJEROS,HOMICIDIO
2,2016-0003,2016-01-03,07:00:00,1,AVENIDA,-58.390403,-34.631894,MOTO,AUTO,HOMICIDIO
3,2016-0004,2016-01-10,00:00:00,8,AVENIDA,-58.465039,-34.680930,MOTO,NaN,HOMICIDIO
4,2016-0005,2016-01-21,05:20:00,1,AVENIDA,-58.387183,-34.622466,MOTO,PASAJEROS,HOMICIDIO
...,...,...,...,...,...,...,...,...,...,...
24476,LC-2021-0652849,2021-12-31,19:30:00,9,AVENIDA,-58.513477,-34.659714,NaN,NaN,LESION
24477,LC-2021-0652865,2021-12-31,19:40:00,9,AVENIDA,-58.488327,-34.641753,NaN,NaN,LESION
24478,LC-2021-0652907,2021-12-31,20:00:00,1,NaN,-58.382894,-34.583083,NaN,NaN,LESION
24479,LC-2021-0652921,2021-12-31,22:00:00,5,CALLE,-58.414532,-34.614288,MOTO,PASAJEROS,LESION


Se detectada un error en el formato de coordenadas y un dato atípico. Revisar calidad de las coordenadas a continuación:

In [50]:
print('pos y máxima: ',Hechos['pos y'].max())
print('pos y mínima: ',Hechos['pos y'].min())
print('pos x máxima: ',Hechos['pos x'].max())
print('pos x mínima: ',Hechos['pos x'].min())

pos y máxima:  -34.53465378
pos y mínima:  -34.72890716
pos x máxima:  -58.29963273
pos x mínima:  -58.53111373207886


#### 5. Exportar a CSV en la carpeta "Datos_Dashboard":

In [51]:
# Hechos.to_csv('Datos_Dashboard/Hechos.csv',index='False')

"""Esto ya no es necesario porque se decide unir el "Homicidios" con "Lesionados"""

'Esto ya no es necesario porque se decide unir el "Homicidios" con "Lesionados'

#### 6. Unir las tablas de víctimas de "Lesiones" y "Homicidios":

In [52]:
# Revisar columnas

print('Homicidios:',V_Homicidios.columns)
print('Lesiones:',V_Lesiones.columns)

Homicidios: Index(['ID_hecho', 'FECHA', 'ROL', 'VICTIMA', 'SEXO', 'EDAD'], dtype='object')
Lesiones: Index(['ID hecho', 'FECHA ', 'VEHICULO_VICTIMA', 'SEXO', 'EDAD_VICTIMA',
       'GRAVEDAD'],
      dtype='object')


Agregar columnas faltantes:

In [53]:
# Agregar columna "TIPO" a cada DataFrame
V_Homicidios['TIPO'] = 'HOMICIDIO'

# Agregar Columna "ROL" a Lesiones
V_Lesiones['ROL'] = 'SD'

Renombrar columnas para que hayan coincidencias entre los dataframes y reorganizarlas:

In [54]:
# Renombrar columnas en V_Lesiones para que coincidan con V_Homicidios
V_Lesiones = V_Lesiones.rename(columns={
    'ID hecho': 'ID_hecho',
    'FECHA ': 'FECHA',
    'VEHICULO_VICTIMA': 'VICTIMA',
    'EDAD_VICTIMA': 'EDAD',
    'GRAVEDAD': 'TIPO',
    })

# Reorganizar las columnas en V_Homicidios
V_Homicidios = V_Homicidios[['ID_hecho', 'FECHA', 'ROL', 'VICTIMA', 'SEXO', 'EDAD', 'TIPO']]

# Concatenar verticalmente los DataFrames
Victimas = pd.concat([V_Homicidios, V_Lesiones], ignore_index=True)

In [55]:
Victimas

,ID_hecho,FECHA,ROL,VICTIMA,SEXO,EDAD,TIPO
0,2016-0001,2016-01-01,CONDUCTOR,MOTO,MASCULINO,19,HOMICIDIO
1,2016-0002,2016-01-02,CONDUCTOR,AUTO,MASCULINO,70,HOMICIDIO
2,2016-0003,2016-01-03,CONDUCTOR,MOTO,MASCULINO,30,HOMICIDIO
3,2016-0004,2016-01-10,CONDUCTOR,MOTO,MASCULINO,18,HOMICIDIO
4,2016-0005,2016-01-21,CONDUCTOR,MOTO,MASCULINO,29,HOMICIDIO
...,...,...,...,...,...,...,...
28317,LC-2021-0451911,2021-09-11,SD,TRANSPORTE PUBLICO,VARON,87.0,LEVE
28318,LC-2021-0530228,2021-10-25,SD,TRANSPORTE PUBLICO,MUJER,60.0,LEVE
28319,LC-2021-0530228,2021-10-25,SD,TRANSPORTE PUBLICO,MUJER,32.0,LEVE
28320,LC-2021-0201378,2021-05-02,SD,MOTO,VARON,32.0,LEVE


#### 7. Dar formato a las columnas e imputar valores faltantes:

In [56]:
# Reemplazar "SD" con NaN en ambos DataFrames
Victimas.replace('SD', np.nan, inplace=True)

# Convertir la columna 'FECHA' al tipo datetime
Victimas['FECHA'] = pd.to_datetime(Victimas['FECHA'], errors='coerce')

# Convertir la columna 'FECHA' al tipo datetime
Victimas['EDAD'] = pd.to_numeric(Victimas['EDAD'], errors='coerce')

Victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28322 entries, 0 to 28321
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   ID_hecho  28322 non-null  object        
 1   FECHA     28322 non-null  datetime64[ns]
 2   ROL       706 non-null    object        
 3   VICTIMA   12041 non-null  object        
 4   SEXO      26088 non-null  object        
 5   EDAD      23798 non-null  float64       
 6   TIPO      28322 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 1.5+ MB


Imputar datos para ROL:

In [57]:
def imputar_rol(row):
    if pd.isnull(row['ROL']) and row['VICTIMA'] == 'PASAJEROS':
        return 'PASAJERO_ACOMPAÑANTE'
    elif pd.isnull(row['ROL']) and pd.isnull(row['VICTIMA']):
        return 'SD'
    elif pd.isnull(row['ROL']) and not pd.isnull(row['VICTIMA']):
        return 'CONDUCTOR'
    else:
        return row['ROL']

# Aplicar la función a la columna 'ROL'
Victimas['ROL'] = Victimas.apply(imputar_rol, axis=1)

Recordando que este campo no existía para las Lesiones y se tratará de imputar a partir de los valores de las víctimas, por ejemplo si el vehículo es de pasajeros lo más probable es que sea un pasajero, si es peatón obviamente es peatón, etc.

In [58]:
Victimas['ROL'].unique()

array(['CONDUCTOR', 'PASAJERO_ACOMPAÑANTE', 'PEATON', 'SD', 'CICLISTA'],
      dtype=object)

In [59]:
Victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28322 entries, 0 to 28321
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   ID_hecho  28322 non-null  object        
 1   FECHA     28322 non-null  datetime64[ns]
 2   ROL       28322 non-null  object        
 3   VICTIMA   12041 non-null  object        
 4   SEXO      26088 non-null  object        
 5   EDAD      23798 non-null  float64       
 6   TIPO      28322 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 1.5+ MB


Desafortunadamente se observa que no se registran muchos datos del "ROL" de la víctima ni del tipo de vehículo en donde se transportaba la víctima. Se presume un sistema muy débil para el registro de esta infromación. Se conservan los datos para el conteo de lesiones, pero se tiene en cuenta que discriminar por estos campos no es relevante. Sin embargo, para las víctimas que terminaron en muerte, estos campos tenían más del 98% de información, lo cual se debe tener presente.

In [60]:
print("Columnas en df_victimas:", Victimas.columns)
print("Columnas en df_hechos:", Hechos.columns)


Columnas en df_victimas: Index(['ID_hecho', 'FECHA', 'ROL', 'VICTIMA', 'SEXO', 'EDAD', 'TIPO'], dtype='object')
Columnas en df_hechos: Index(['ID', 'FECHA', 'HORA', 'COMUNA', 'TIPO_DE_CALLE', 'pos x', 'pos y',
       'VICTIMA', 'ACUSADO', 'TIPO'],
      dtype='object')


#### 8. Combinar los dataframe de "Hechos" con "Vícimas":

Esto ayudará a los gráficos en el dashboard y la creación de los KPI's para medir el desempeños de las políticas públicas (planes de acción del gobierno) para disminuir el índice de mortalidad por accidentes viales:

8.1. Corrección de datos y unión de dataframes:

In [61]:
# Eliminar las columnas "VICTIMA" y "FECHA" del DataFrame de víctimas
df_victimas = Victimas.drop(['VICTIMA','FECHA'], axis=1)

Hechos = Hechos.drop(['TIPO'], axis=1)

# Unir los dos DataFrames utilizando el ID_hecho como clave
Siniestros_Viales = pd.merge(df_victimas, Hechos, left_on='ID_hecho', right_on='ID')

# Eliminar la columna "ID" del DataFrame de víctimas
Siniestros_Viales = Siniestros_Viales.drop(['ID'], axis=1)

# Corregir valores de "SEXO" ("Varon por Masculino y Mujer por Femenino")
Siniestros_Viales.loc[Siniestros_Viales['SEXO'] == 'VARON', 'SEXO'] = 'MASCULINO'
Siniestros_Viales.loc[Siniestros_Viales['SEXO'] == 'MUJER', 'SEXO'] = 'FEMENINO'

# Renombrar las columna de "ID_Hecho"
Siniestros_Viales = Siniestros_Viales.rename(columns={'ID_hecho': 'ID'})

# ver nuevo dataframe de "Siniestros_Viales"
print(Siniestros_Viales.info())
Siniestros_Viales


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28315 entries, 0 to 28314
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             28315 non-null  object        
 1   ROL            28315 non-null  object        
 2   SEXO           26081 non-null  object        
 3   EDAD           23791 non-null  float64       
 4   TIPO           28315 non-null  object        
 5   FECHA          28315 non-null  datetime64[ns]
 6   HORA           28311 non-null  object        
 7   COMUNA         26689 non-null  object        
 8   TIPO_DE_CALLE  15436 non-null  object        
 9   pos x          26888 non-null  float64       
 10  pos y          26888 non-null  float64       
 11  VICTIMA        15484 non-null  object        
 12  ACUSADO        10374 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(9)
memory usage: 2.8+ MB
None


,ID,ROL,SEXO,EDAD,TIPO,FECHA,HORA,COMUNA,TIPO_DE_CALLE,pos x,pos y,VICTIMA,ACUSADO
0,2016-0001,CONDUCTOR,MASCULINO,19.0,HOMICIDIO,2016-01-01,04:00:00,8,AVENIDA,-58.475340,-34.687570,MOTO,AUTO
1,2016-0002,CONDUCTOR,MASCULINO,70.0,HOMICIDIO,2016-01-02,01:15:00,9,GRAL PAZ,-58.508775,-34.669777,AUTO,PASAJEROS
2,2016-0003,CONDUCTOR,MASCULINO,30.0,HOMICIDIO,2016-01-03,07:00:00,1,AVENIDA,-58.390403,-34.631894,MOTO,AUTO
3,2016-0004,CONDUCTOR,MASCULINO,18.0,HOMICIDIO,2016-01-10,00:00:00,8,AVENIDA,-58.465039,-34.680930,MOTO,NaN
4,2016-0005,CONDUCTOR,MASCULINO,29.0,HOMICIDIO,2016-01-21,05:20:00,1,AVENIDA,-58.387183,-34.622466,MOTO,PASAJEROS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28310,LC-2021-0451911,CONDUCTOR,MASCULINO,87.0,LEVE,2021-09-11,18:00:00,14,AVENIDA,-58.420119,-34.581370,PASAJEROS,PASAJEROS
28311,LC-2021-0530228,CONDUCTOR,FEMENINO,60.0,LEVE,2021-10-25,12:00:00,14,NaN,-58.406897,-34.581142,PASAJEROS,PASAJEROS
28312,LC-2021-0530228,CONDUCTOR,FEMENINO,32.0,LEVE,2021-10-25,12:00:00,14,NaN,-58.406897,-34.581142,PASAJEROS,PASAJEROS
28313,LC-2021-0201378,CONDUCTOR,MASCULINO,32.0,LEVE,2021-05-02,00:00:00,NaN,NaN,NaN,NaN,MOTO,OBJETO FIJO


#### 9. Crear columnas auxiliares para el cálculo de KPI's:

Se decide crear columnas que faciliten las uniones con los datos población para los KPI's.

In [62]:
# Crear las columnas AÑO, MES y SEMESTRE
Siniestros_Viales['AÑO'] = Siniestros_Viales['FECHA'].dt.year.astype(int)
Siniestros_Viales['MES'] = Siniestros_Viales['FECHA'].dt.month
Siniestros_Viales['SEMESTRE'] = Siniestros_Viales['FECHA'].dt.month.apply(lambda x: 1 if x <= 6 else 2)

# Crear la columna Aux_Sem
Siniestros_Viales['Aux_Sem'] = Siniestros_Viales['AÑO'].astype(str) + '-' + Siniestros_Viales['COMUNA'] + '-' + Siniestros_Viales['SEMESTRE'].astype(str)

# Mostrar el DataFrame resultante
Siniestros_Viales

,ID,ROL,SEXO,EDAD,TIPO,FECHA,HORA,COMUNA,TIPO_DE_CALLE,pos x,pos y,VICTIMA,ACUSADO,AÑO,MES,SEMESTRE,Aux_Sem
0,2016-0001,CONDUCTOR,MASCULINO,19.0,HOMICIDIO,2016-01-01,04:00:00,8,AVENIDA,-58.475340,-34.687570,MOTO,AUTO,2016,1,1,2016-8-1
1,2016-0002,CONDUCTOR,MASCULINO,70.0,HOMICIDIO,2016-01-02,01:15:00,9,GRAL PAZ,-58.508775,-34.669777,AUTO,PASAJEROS,2016,1,1,2016-9-1
2,2016-0003,CONDUCTOR,MASCULINO,30.0,HOMICIDIO,2016-01-03,07:00:00,1,AVENIDA,-58.390403,-34.631894,MOTO,AUTO,2016,1,1,2016-1-1
3,2016-0004,CONDUCTOR,MASCULINO,18.0,HOMICIDIO,2016-01-10,00:00:00,8,AVENIDA,-58.465039,-34.680930,MOTO,NaN,2016,1,1,2016-8-1
4,2016-0005,CONDUCTOR,MASCULINO,29.0,HOMICIDIO,2016-01-21,05:20:00,1,AVENIDA,-58.387183,-34.622466,MOTO,PASAJEROS,2016,1,1,2016-1-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28310,LC-2021-0451911,CONDUCTOR,MASCULINO,87.0,LEVE,2021-09-11,18:00:00,14,AVENIDA,-58.420119,-34.581370,PASAJEROS,PASAJEROS,2021,9,2,2021-14-2
28311,LC-2021-0530228,CONDUCTOR,FEMENINO,60.0,LEVE,2021-10-25,12:00:00,14,NaN,-58.406897,-34.581142,PASAJEROS,PASAJEROS,2021,10,2,2021-14-2
28312,LC-2021-0530228,CONDUCTOR,FEMENINO,32.0,LEVE,2021-10-25,12:00:00,14,NaN,-58.406897,-34.581142,PASAJEROS,PASAJEROS,2021,10,2,2021-14-2
28313,LC-2021-0201378,CONDUCTOR,MASCULINO,32.0,LEVE,2021-05-02,00:00:00,NaN,NaN,NaN,NaN,MOTO,OBJETO FIJO,2021,5,1,NaN


In [63]:
Siniestros_Viales['VICTIMA'].unique()

array(['MOTO', 'AUTO', 'PEATON', nan, 'CARGAS', 'BICICLETA', 'PASAJEROS',
       'MOVIL', 'OBJETO FIJO', 'PEATON_MOTO', 'OTRO', 'MULTIPLE'],
      dtype=object)

In [64]:
# Filtra los datos donde 'VICTIMA' es igual a "MULTIPLE"
datos_multiple = Siniestros_Viales[Siniestros_Viales['VICTIMA'] == 'MULTIPLE']

# Muestra los datos filtrados
datos_multiple

,ID,ROL,SEXO,EDAD,TIPO,FECHA,HORA,COMUNA,TIPO_DE_CALLE,pos x,pos y,VICTIMA,ACUSADO,AÑO,MES,SEMESTRE,Aux_Sem
13872,LC-2020-0526083,CONDUCTOR,FEMENINO,74.0,LEVE,2020-09-18,12:40:00 p.m.,10,CALLE,-58.492919,-34.623983,MULTIPLE,PASAJEROS,2020,9,2,2020-10-2
13981,LC-2020-0480830,CONDUCTOR,MASCULINO,20.0,LEVE,2020-08-15,12:00:00 a.m.,14,CALLE,-58.421238,-34.586964,MULTIPLE,AUTO,2020,8,2,2020-14-2
13982,LC-2020-0480830,CONDUCTOR,MASCULINO,19.0,LEVE,2020-08-15,12:00:00 a.m.,14,CALLE,-58.421238,-34.586964,MULTIPLE,AUTO,2020,8,2,2020-14-2
14188,LC-2020-0168559,CONDUCTOR,MASCULINO,29.0,LEVE,2020-03-16,03:41:00 p.m.,5,AVENIDA,-58.416101,-34.625436,MULTIPLE,AUTO,2020,3,1,2020-5-1
14189,LC-2020-0168559,CONDUCTOR,MASCULINO,44.0,LEVE,2020-03-16,03:41:00 p.m.,5,AVENIDA,-58.416101,-34.625436,MULTIPLE,AUTO,2020,3,1,2020-5-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28116,LC-2021-0306122,CONDUCTOR,FEMENINO,50.0,LEVE,2021-06-29,13:30:00,15,AVENIDA,-58.498384,-34.596368,MULTIPLE,AUTO,2021,6,1,2021-15-1
28117,LC-2021-0306122,CONDUCTOR,MASCULINO,43.0,LEVE,2021-06-29,13:30:00,15,AVENIDA,-58.498384,-34.596368,MULTIPLE,AUTO,2021,6,1,2021-15-1
28132,LC-2021-0402497,CONDUCTOR,MASCULINO,36.0,LEVE,2021-08-14,01:30:00,15,CALLE,-58.498170,-34.591036,MULTIPLE,AUTO,2021,8,2,2021-15-2
28133,LC-2021-0402497,CONDUCTOR,NaN,NaN,LEVE,2021-08-14,01:30:00,15,CALLE,-58.498170,-34.591036,MULTIPLE,AUTO,2021,8,2,2021-15-2


#### 10. Exportar a CSV a la carpeta "Datos_Dashboard":

Antes de exportar, durante el desarrollo del EDA, surgieron nuevos requerimientos de transformación, en especial de coordenadas, ya que en la transformación de coordenadas plana en mundiales, surgío una desviación sistemática corregida con el promedio del error promedio de los datos conocidos, lo cual se aprecia en el EDA funcionó para corregir correctamente las coordenadas. (Esto para los datos originados en Lesiones ya que en Homicidios no se tenía este problema)

In [65]:
# Se requiere una última modificación detectada en el EDA:

# Condiciones para reemplazar en la columna 'ROL'
condiciones = (Siniestros_Viales['VICTIMA'] == 'BICICLETA') | (Siniestros_Viales['VICTIMA'] == 'PEATON')

# Aplicar el reemplazo en la columna 'ROL' solo para las filas que cumplen las condiciones
Siniestros_Viales.loc[condiciones, 'ROL'] = Siniestros_Viales.loc[condiciones, 'VICTIMA']

# Luego, realiza la sustitución específica para 'BICICLETA' y 'PEATON'
Siniestros_Viales['ROL'] = Siniestros_Viales['ROL'].replace({'BICICLETA': 'CICLISTA', 'PEATON': 'PEATON'})

# -----------------------------------------------------------------------------------------------------------------------------------

# Diccionario de Victimas
mapeo_victimas = {'OBJETO FIJO': 'PEATON', 'PEATON_MOTO': 'PEATON', 'MULTIPLE':None}

# Aplicar el reemplazo en la columna 'VICTIMA'
Siniestros_Viales['VICTIMA'] = Siniestros_Viales['VICTIMA'].replace(mapeo_victimas)

valores_unicos = Siniestros_Viales['VICTIMA'].unique()

print(valores_unicos)

# Crear rango Etario ------------------------------------------------------------------------------------------------------------------------

# Define una función para asignar las categorías de R_Etario
def asignar_r_etario(edad):
    if 0 <= edad <= 5:
        return 'Primera Infancia'
    elif 6 <= edad <= 11:
        return 'Infancia'
    elif 12 <= edad <= 18:
        return 'Adolescencia'
    elif 19 <= edad <= 26:
        return 'Juventud'
    elif 27 <= edad <= 59:
        return 'Adultez'
    elif edad >= 60:
        return 'Vejez'
    else:
        return 'Desconocido'

# Aplica la función a la columna 'EDAD' y crea la columna 'R_Etario'
Siniestros_Viales['R_Etario'] = Siniestros_Viales['EDAD'].apply(asignar_r_etario)


# Exportar a CSV ------------------------------------------------------------------------------------------------------------------------------

# Definir el orden de las columnas
column_order = ['ID', 'FECHA', 'AÑO', 'MES', 'HORA', 'SEMESTRE', 'Aux_Sem', 'VICTIMA', 'ACUSADO', 'TIPO', 'ROL', 'SEXO', 'EDAD', 'COMUNA', 'TIPO_DE_CALLE', 'pos x', 'pos y','R_Etario' ]

# Seleccionar el orden deseado
Siniestros_Viales = Siniestros_Viales[column_order]

# Guarda el DataFrame en un archivo CSV
Siniestros_Viales.to_csv('Datos_Dashboard/Siniestros_Viales.csv', index=False)

# Muestra la información del DataFrame
Siniestros_Viales.info()

['MOTO' 'AUTO' 'PEATON' nan 'CARGAS' 'BICICLETA' 'PASAJEROS' 'MOVIL'
 'OTRO' None]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28315 entries, 0 to 28314
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             28315 non-null  object        
 1   FECHA          28315 non-null  datetime64[ns]
 2   AÑO            28315 non-null  int32         
 3   MES            28315 non-null  int32         
 4   HORA           28311 non-null  object        
 5   SEMESTRE       28315 non-null  int64         
 6   Aux_Sem        26689 non-null  object        
 7   VICTIMA        15371 non-null  object        
 8   ACUSADO        10374 non-null  object        
 9   TIPO           28315 non-null  object        
 10  ROL            28315 non-null  object        
 11  SEXO           26081 non-null  object        
 12  EDAD           23791 non-null  float64       
 13  COMUNA         26689 non-null  object        
 14  TIPO_DE_CALLE  15436 non-null  object        
 15  pos x          2688

Ya se tiene el documneto listo para usar en la herramienta de visualización con todos los datos arreglados, estructura transformada, valores imputados y los tipos de datos correctos.

#### 11. Datos de Población:

Se consigue inicialmente los datos del Censo de 2010 y el 2022, se usó en la operación de ETL un prorrrateo con la variación pobalcional por comuna, para crear una base anual supuniendo un crecimiento lineal. Pero Uno de los KPI's requiere seguimiento semestral, así que se transformarán los datos para crear una base con población semestral por comuna. Luego a partir de este datafrome se traerán los datos necesarios del archivo de Siestros Viales para analizar estos datos según la Población:

#### 11.1. Traer datos ordenados y cread columnas auxiliares para transformar la tabla a semestres:

In [66]:
# Traer datos ordenados por Comuna y Año
Poblacion_Semestre = Poblacion.sort_values(by=['Comuna', 'Año'])

Poblacion_Semestre

,Comuna,Año,Población
0,1,2010,205886.0
15,1,2011,207358.0
30,1,2012,208831.0
45,1,2013,210303.0
60,1,2014,211775.0
...,...,...,...
134,15,2018,192109.0
149,15,2019,193301.0
164,15,2020,194492.0
179,15,2021,195684.0


Se crean los campos Población_1 y Variación:

In [67]:
# Crear la columna auxiliar "Poblacion_1"
Poblacion_Semestre['Poblacion_1'] = Poblacion_Semestre['Población'].shift(1)

# Crear la columna auxiliar "Variacion"
Poblacion_Semestre['Variacion'] = (Poblacion_Semestre['Poblacion_1'] - Poblacion_Semestre['Población']).div(2)

# Llenar la columna "Poblacion_1" restando la "Variacion"
Poblacion_Semestre['Poblacion_1'] = Poblacion_Semestre['Poblacion_1'].sub(Poblacion_Semestre['Variacion'], fill_value=0)

# Eliminar la columna "ID" del DataFrame de víctimas
Poblacion_Semestre = Poblacion_Semestre.drop(['Variacion'], axis=1)

# Mostrar el DataFrame resultante
Poblacion_Semestre

,Comuna,Año,Población,Poblacion_1
0,1,2010,205886.0,NaN
15,1,2011,207358.0,206622.0
30,1,2012,208831.0,208094.5
45,1,2013,210303.0,209567.0
60,1,2014,211775.0,211039.0
...,...,...,...,...
134,15,2018,192109.0,191513.0
149,15,2019,193301.0,192705.0
164,15,2020,194492.0,193896.5
179,15,2021,195684.0,195088.0


#### 11.2. Creamos registros duplicados y generamos datos semestrales de población:

In [68]:
# 1. Duplicar cada fila
Poblacion_Semestre = pd.concat([Poblacion_Semestre]*2, ignore_index=True)

# 2. Crear la columna 'Semestre' con valores 1 y 2
Poblacion_Semestre['Semestre'] = (Poblacion_Semestre.groupby('Año').cumcount() % 2) + 1

# 3. Establecer como NaN los valores de la columna 'Población' para los semestres 1
Poblacion_Semestre.loc[Poblacion_Semestre['Semestre'] == 1, 'Población'] = None

# 4. Imputar los valores NaN en la columna 'Población' para los semestres 1 con los valores de 'Poblacion_1'
Poblacion_Semestre.loc[Poblacion_Semestre['Semestre'] == 1, 'Población'] = Poblacion_Semestre['Poblacion_1']

# 5. Eliminar la columna 'Poblacion_1'
Poblacion_Semestre = Poblacion_Semestre.drop(columns=['Poblacion_1'])

Observamos que queden correctas las imputaciones por comuna y semestre:

In [69]:
# Filtrar los registros donde el valor en la columna "Comuna" es igual a 1
registros_comuna_1 = Poblacion_Semestre[Poblacion_Semestre['Comuna'] == 1]

# Mostrar los registros resultantes
print(registros_comuna_1)

     Comuna   Año  Población  Semestre
0         1  2010        NaN         1
1         1  2011   206622.0         1
2         1  2012   208094.5         1
3         1  2013   209567.0         1
4         1  2014   211039.0         1
5         1  2015   212511.5         1
6         1  2016   213984.0         1
7         1  2017   215456.0         1
8         1  2018   216928.5         1
9         1  2019   218401.0         1
10        1  2020   219873.0         1
11        1  2021   221345.5         1
12        1  2022   222818.0         1
195       1  2010   205886.0         2
196       1  2011   207358.0         2
197       1  2012   208831.0         2
198       1  2013   210303.0         2
199       1  2014   211775.0         2
200       1  2015   213248.0         2
201       1  2016   214720.0         2
202       1  2017   216192.0         2
203       1  2018   217665.0         2
204       1  2019   219137.0         2
205       1  2020   220609.0         2
206       1  2021   22208

#### 11.3. Creamos una columna auxiliar de semestre que nos servirá para unir datos agragados desde la hoja de "Siniestros":

In [70]:
# Crear la columna Aux_Sem
Poblacion_Semestre['Aux_Sem'] = Poblacion_Semestre['Año'
    ].astype(str) + '-' + Poblacion_Semestre['Comuna'].astype(str) + '-' + Poblacion_Semestre['Semestre'].astype(str)

# Definir el orden de las columnas
order = ['Año','Semestre', 'Aux_Sem', 'Comuna',  'Población']

# Seleccionar el orden deseado
Poblacion_Semestre = Poblacion_Semestre[order]

# Mostrar el DataFrame resultante
Poblacion_Semestre

,Año,Semestre,Aux_Sem,Comuna,Población
0,2010,1,2010-1-1,1,NaN
1,2011,1,2011-1-1,1,206622.0
2,2012,1,2012-1-1,1,208094.5
3,2013,1,2013-1-1,1,209567.0
4,2014,1,2014-1-1,1,211039.0
...,...,...,...,...,...
385,2018,2,2018-15-2,15,192109.0
386,2019,2,2019-15-2,15,193301.0
387,2020,2,2020-15-2,15,194492.0
388,2021,2,2021-15-2,15,195684.0


#### 11.4. Crear las columnas de "No. Muertes" y "No. Lesiones" basado en los datos de "Homicidios":

In [71]:
# Agrupar y contar en Siniestros_Viales
conteo_tipos = Siniestros_Viales.groupby(['Aux_Sem', 'TIPO']).size().unstack(fill_value=0)

# Filtrar solo las columnas relevantes ('HOMICIDIO', 'GRAVE', 'LEVE')
conteo_tipos = conteo_tipos[['HOMICIDIO', 'GRAVE', 'LEVE']]

# Fusionar con Poblacion_Semestre
Poblacion_Semestre = Poblacion_Semestre.merge(conteo_tipos, how='left', left_on='Aux_Sem', right_index=True)

# Renombrar columnas
Poblacion_Semestre = Poblacion_Semestre.rename(columns={'HOMICIDIO': 'No. Muertes', 'GRAVE': 'No. Graves', 'LEVE': 'No. Leves'})

# Imputar valores nulos como cero "0"
Poblacion_Semestre[['No. Muertes', 'No. Graves', 'No. Leves']] = Poblacion_Semestre[['No. Muertes', 'No. Graves', 'No. Leves']].fillna(0)

# Mostrar el resultado
print(Poblacion_Semestre)


      Año  Semestre    Aux_Sem  Comuna  Población  No. Muertes  No. Graves  \
0    2010         1   2010-1-1       1        NaN          0.0         0.0   
1    2011         1   2011-1-1       1   206622.0          0.0         0.0   
2    2012         1   2012-1-1       1   208094.5          0.0         0.0   
3    2013         1   2013-1-1       1   209567.0          0.0         0.0   
4    2014         1   2014-1-1       1   211039.0          0.0         0.0   
..    ...       ...        ...     ...        ...          ...         ...   
385  2018         2  2018-15-2      15   192109.0          3.0         0.0   
386  2019         2  2019-15-2      15   193301.0          7.0         0.0   
387  2020         2  2020-15-2      15   194492.0          4.0         3.0   
388  2021         2  2021-15-2      15   195684.0          0.0        13.0   
389  2022         2  2022-15-2      15   196876.0          0.0         0.0   

     No. Leves  
0          0.0  
1          0.0  
2          0

In [72]:
Siniestros_Viales['TIPO'].unique()

array(['HOMICIDIO', 'LEVE', 'GRAVE'], dtype=object)

#### 11.5. Crear THSV (tasa de homicidios de Siniestros Viales)
(Número de homicidios en siniestros viales / Población total) * 100,000

In [73]:
# Ordenar el DataFrame por Año, Semestre y Comuna
Poblacion_Semestre = Poblacion_Semestre.sort_values(by=['Comuna', 'Año', 'Semestre'])

# Calcular la columna THSV
Poblacion_Semestre['THSV'] = (Poblacion_Semestre['No. Muertes'] / Poblacion_Semestre['Población']) * 100000

# Crear la columna THSV_1 usando shift para obtener el valor del registro inmediatamente anterior
Poblacion_Semestre['THSV_1'] = Poblacion_Semestre['THSV'].shift(1)

# Llenar con 0 en los casos donde no haya un valor anterior (primer semestre del primer año)
Poblacion_Semestre['THSV_1'] = Poblacion_Semestre['THSV_1'].fillna(0)


In [74]:
Poblacion_Semestre[Poblacion_Semestre['THSV_1'] == 0]


,Año,Semestre,Aux_Sem,Comuna,Población,No. Muertes,No. Graves,No. Leves,THSV,THSV_1
0,2010,1,2010-1-1,1,NaN,0.0,0.0,0.0,NaN,0.0
195,2010,2,2010-1-2,1,205886.0,0.0,0.0,0.0,0.000000,0.0
1,2011,1,2011-1-1,1,206622.0,0.0,0.0,0.0,0.000000,0.0
196,2011,2,2011-1-2,1,207358.0,0.0,0.0,0.0,0.000000,0.0
2,2012,1,2012-1-1,1,208094.5,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...
187,2015,1,2015-15-1,15,187937.0,0.0,0.0,0.0,0.000000,0.0
382,2015,2,2015-15-2,15,188533.0,0.0,0.0,0.0,0.000000,0.0
188,2016,1,2016-15-1,15,189129.0,3.0,0.0,0.0,1.586219,0.0
194,2022,1,2022-15-1,15,196280.0,0.0,0.0,0.0,0.000000,0.0


In [75]:
# Crear Columna "PR_THSV" (porcentaje de reducción de la tasa de homicidios de siniestros viales)
Poblacion_Semestre['PR_THSV'] = 100*(Poblacion_Semestre['THSV'] - Poblacion_Semestre['THSV_1'])/Poblacion_Semestre['THSV_1']

Eliminar columna auxiliar:

In [76]:
# Eliminar la columna "THSV_1" del DataFrame de Poblacion_Semestre

# Filtrar el DataFrame
filtro_poblacion = (Poblacion_Semestre['Año'].between(2016, 2021)) & (Poblacion_Semestre['Comuna'] == 15)# & (Poblacion_Semestre['THSV_1'] == 0)
datos_filtrados = Poblacion_Semestre[filtro_poblacion]

print(datos_filtrados[['THSV','THSV_1','Año','Semestre','Comuna']] )

#Poblacion_Semestre = Poblacion_Semestre.drop(['THSV_1'], axis=1)

         THSV    THSV_1   Año  Semestre  Comuna
188  1.586219  0.000000  2016         1      15
383  2.635393  1.586219  2016         2      15
189  1.050856  2.635393  2017         1      15
384  1.571363  1.050856  2017         2      15
190  4.177262  1.571363  2018         1      15
385  1.561613  4.177262  2018         2      15
191  1.037856  1.561613  2019         1      15
386  3.621295  1.037856  2019         2      15
192  1.547217  3.621295  2020         1      15
387  2.056640  1.547217  2020         2      15
193  2.562946  2.056640  2021         1      15
388  0.000000  2.562946  2021         2      15


#### 11.5. Crear Indice de Mortalidad de Motociclistas con respecto del año anterior:

Contamos las muertes donde la víctima se relaciona con "MOTO" en el tipo de vehículo de la víctima.

In [77]:
# Contar el número de registros en Siniestros_Viales que cumplen con las condiciones
count_m_moto = Siniestros_Viales.loc[
    (Siniestros_Viales['Aux_Sem'].isin(Poblacion_Semestre['Aux_Sem'])) &
    (Siniestros_Viales['TIPO'] == 'HOMICIDIO') &
    (Siniestros_Viales['VICTIMA'] == 'MOTO')
].groupby('Aux_Sem')['TIPO'].count().reset_index()

# Renombrar la columna y fusionar con Poblacion_Semestre
count_m_moto = count_m_moto.rename(columns={'TIPO': 'M. Moto'})
Poblacion_Semestre = pd.merge(Poblacion_Semestre, count_m_moto, on='Aux_Sem', how='left')

# Rellenar NaN con 0 si no hay coincidencias
Poblacion_Semestre['M. Moto'] = Poblacion_Semestre['M. Moto'].fillna(0).astype(int)

# Mostrar el DataFrame resultante
Poblacion_Semestre

,Año,Semestre,Aux_Sem,Comuna,Población,No. Muertes,No. Graves,No. Leves,THSV,THSV_1,PR_THSV,M. Moto
0,2010,1,2010-1-1,1,NaN,0.0,0.0,0.0,NaN,0.000000,NaN,0
1,2010,2,2010-1-2,1,205886.0,0.0,0.0,0.0,0.000000,0.000000,NaN,0
2,2011,1,2011-1-1,1,206622.0,0.0,0.0,0.0,0.000000,0.000000,NaN,0
3,2011,2,2011-1-2,1,207358.0,0.0,0.0,0.0,0.000000,0.000000,NaN,0
4,2012,1,2012-1-1,1,208094.5,0.0,0.0,0.0,0.000000,0.000000,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
385,2020,2,2020-15-2,15,194492.0,4.0,3.0,299.0,2.056640,1.547217,32.925090,4
386,2021,1,2021-15-1,15,195088.0,5.0,9.0,253.0,2.562946,2.056640,24.618121,2
387,2021,2,2021-15-2,15,195684.0,0.0,13.0,387.0,0.000000,2.562946,-100.000000,0
388,2022,1,2022-15-1,15,196280.0,0.0,0.0,0.0,0.000000,0.000000,NaN,0


In [78]:
# Ordenar el DataFrame por Año, Semestre, Comuna y moto
Poblacion_Semestre = Poblacion_Semestre.sort_values(by=['Comuna', 'Año', 'Semestre'])

# Crear la columna THSV_1 usando shift para obtener el valor del registro inmediatamente anterior
Poblacion_Semestre['M. Moto_1'] = Poblacion_Semestre['M. Moto'].shift(1)

# Llenar con 0 en los casos donde no haya un valor anterior (primer semestre del primer año)
Poblacion_Semestre['M. Moto_1'] = Poblacion_Semestre['M. Moto_1'].fillna(0)

# Crear Columna "I. Moto"
Poblacion_Semestre['I. MOTO'] = 100 * (Poblacion_Semestre['M. Moto'] - Poblacion_Semestre['M. Moto_1']) / Poblacion_Semestre['M. Moto_1']

In [79]:
# Suponiendo que 'df' es tu DataFrame
df_filtrado = Poblacion_Semestre.loc[Poblacion_Semestre['Año'] == 2016]

# Mostrar el DataFrame filtrado
df_filtrado.head()

,Año,Semestre,Aux_Sem,Comuna,Población,No. Muertes,No. Graves,No. Leves,THSV,THSV_1,PR_THSV,M. Moto,M. Moto_1,I. MOTO
12,2016,1,2016-1-1,1,213984.0,12.0,0.0,0.0,5.607896,0.000000,inf,4,0.0,inf
13,2016,2,2016-1-2,1,214720.0,10.0,0.0,0.0,4.657228,5.607896,-16.95231,6,4.0,50.0
38,2016,1,2016-2-1,2,159633.5,1.0,0.0,0.0,0.626435,0.000000,inf,1,0.0,inf
39,2016,2,2016-2-2,2,159788.0,0.0,0.0,0.0,0.000000,0.626435,-100.00000,0,1.0,-100.0
64,2016,1,2016-3-1,3,191525.5,4.0,0.0,0.0,2.088495,0.000000,inf,3,0.0,inf


#### 11.6. Eliminar las filas innecesarias:

In [80]:
# Eliminar las filas correspondientes al año 2022
Poblacion_Semestre = Poblacion_Semestre.loc[Poblacion_Semestre['Año'] != 2022]
Poblacion_Semestre = Poblacion_Semestre.query('Año > 2015')

# Imputar valores infinitos con cero en las columnas PR_THSV e I. MOTO
Poblacion_Semestre['PR_THSV'] = Poblacion_Semestre['PR_THSV'].replace([np.inf, -np.inf], 0)
Poblacion_Semestre['I. MOTO'] = Poblacion_Semestre['I. MOTO'].replace([np.inf, -np.inf], 0)

Poblacion_Semestre

,Año,Semestre,Aux_Sem,Comuna,Población,No. Muertes,No. Graves,No. Leves,THSV,THSV_1,PR_THSV,M. Moto,M. Moto_1,I. MOTO
12,2016,1,2016-1-1,1,213984.0,12.0,0.0,0.0,5.607896,0.000000,0.000000,4,0.0,0.000000
13,2016,2,2016-1-2,1,214720.0,10.0,0.0,0.0,4.657228,5.607896,-16.952310,6,4.0,50.000000
14,2017,1,2017-1-1,1,215456.0,13.0,0.0,0.0,6.033715,4.657228,29.555919,3,6.0,-50.000000
15,2017,2,2017-1-2,1,216192.0,7.0,0.0,0.0,3.237863,6.033715,-46.337159,1,3.0,-66.666667
16,2018,1,2018-1-1,1,216928.5,9.0,0.0,0.0,4.148832,3.237863,28.134912,3,1.0,200.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,2019,2,2019-15-2,15,193301.0,7.0,0.0,469.0,3.621295,1.037856,248.920854,3,2.0,50.000000
384,2020,1,2020-15-1,15,193896.5,3.0,2.0,250.0,1.547217,3.621295,-57.274481,0,3.0,-100.000000
385,2020,2,2020-15-2,15,194492.0,4.0,3.0,299.0,2.056640,1.547217,32.925090,4,0.0,0.000000
386,2021,1,2021-15-1,15,195088.0,5.0,9.0,253.0,2.562946,2.056640,24.618121,2,4.0,-50.000000


Para los análisis se debe tener en cuenta que la base de datos de lesiones está desde el año 2019 mientras que la base de datos de muertes desde el 2016.

Relacionar el número de muertes y lesiones por año y por comuna en el dataframe de Población:

#### 11.7. Exportar a CSV en la carpeta de "Datos_Dashboard"

In [81]:
Poblacion_Semestre.to_csv('Datos_Dashboard/Poblacion.csv',index='False')